In [2]:
import pandas as pd
import funciones.test_func as tf
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

host=os.getenv("DB_HOST2")
port=os.getenv("DB_PORT2")
database=os.getenv("DB_NAME2")
user=os.getenv("DB_USER2")
password=os.getenv("DB_PASS2")

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

tables = tf.excecute_query_tables(engine)

dfs = tf.create_df_variable(tables, engine)
transactions = dfs['transactions']
customers= dfs['customers']
products = dfs['products']

In [3]:
transactions

,transaction_id,customer_id,product_id,transaction_date,amount,type,payment_method
0,1,101,201,2023-01-05,200,credit,card
1,2,102,202,2023-01-07,-100,debit,paypal
2,3,101,201,2023-01-15,500,credit,transfer
3,4,103,203,2023-01-20,1200,credit,card
4,5,102,204,2023-02-02,-50,debit,paypal
5,6,101,205,2023-02-14,-300,debit,card
6,7,104,202,2023-02-18,400,credit,cash
7,8,103,206,2023-03-01,-200,debit,cash
8,9,102,207,2023-03-02,1000,credit,transfer
9,10,101,201,2023-03-10,-100,debit,paypal


In [4]:
customers

,customer_id,name,age,country,signup_date
0,101,Alice,29,USA,2022-11-10
1,102,Bob,35,Canada,2022-12-15
2,103,Charlie,42,USA,2023-01-01
3,104,Diana,31,UK,2023-02-10


In [5]:
products

,product_id,product_name,category,price
0,201,Subscription A,subscription,200
1,202,E-Book,digital,100
2,203,Online Course,education,1200
3,204,Gift Card,gift,50
4,205,Consulting Call,service,300
5,206,Webinar,event,200
6,207,Premium Upgrade,upgrade,1000


Challenge #2: Monthly revenue trend for the store
Problem Statement:

Calculate the total net revenue per month from all transactions.
Only include credit transactions.

Requirements:
Output should include:

month (e.g., '2023-01')

total_credit_amount

Sort by month in ascending order.

Use either SQL or pandas.

Optional: If you're using pandas, format transaction_date properly if it's not already a datetime.

In [6]:
#SQL 

query = """
SELECT TO_CHAR(CAST(transaction_date AS date), 'YYYY-MM') AS month_trans, type, SUM(amount) AS total_credit_amount
FROM transactions
WHERE type = 'credit' 
GROUP BY TO_CHAR(CAST(transaction_date AS date), 'YYYY-MM'), type
ORDER BY month_trans
"""

tf.excecute_query(query, engine)

,month_trans,type,total_credit_amount
0,2023-01,credit,1900.0
1,2023-02,credit,400.0
2,2023-03,credit,1000.0


In [7]:
#Python

transactions_credit = transactions[transactions['type'] == 'credit'].copy()
transactions_credit['transaction_date'] = pd.to_datetime(transactions_credit['transaction_date'], format = '%Y-%m-%d', errors='coerce')
transactions_credit['transaction_date_month'] = transactions_credit['transaction_date'].dt.strftime('%Y-%m')
transactions_credit_grouped = transactions_credit.groupby(['transaction_date_month','type']).agg({'amount':'sum'}).reset_index().sort_values(by='transaction_date_month', ascending=True)
transactions_credit_grouped.rename(columns={'amount':'total_credit_amount'}, inplace=True)
transactions_credit_grouped

,transaction_date_month,type,total_credit_amount
0,2023-01,credit,1900
1,2023-02,credit,400
2,2023-03,credit,1000
